# Classification with ConvNeXt and OpenVINO
The [`torchvision.models`](https://pytorch.org/vision/stable/models.html) subpackage contains definitions of models for addressing different tasks, including: image classification, pixelwise semantic segmentation, object detection, instance segmentation, person keypoint detection, video classification, and optical flow. Throughout this notebook we will show how to use one of them.

The ConvNeXt model is based on the [A ConvNet for the 2020s](https://arxiv.org/abs/2201.03545) paper. The outcome of this exploration is a family of pure ConvNet models dubbed ConvNeXt. Constructed entirely from standard ConvNet modules, ConvNeXts compete favorably with Transformers in terms of accuracy and scalability, achieving 87.8% ImageNet top-1 accuracy and outperforming Swin Transformers on COCO detection and ADE20K segmentation, while maintaining the simplicity and efficiency of standard ConvNets.
The `torchvision.models` subpackage [contains](https://pytorch.org/vision/main/models/convnext.html) several pretrained ConvNeXt model. In this tutorial we will use ConvNeXt Tiny model.

#### Table of contents:
- [Prerequisites](#Prerequisites)
- [Get a test image](#Get-a-test-image)
- [Get a pretrained model](#Get-a-pretrained-model)
- [Define a preprocessing and prepare an input data](#Define-a-preprocessing-and-prepare-an-input-data)
- [Use the original model to run an inference](#Use-the-original-model-to-run-an-inference)
- [Convert the model to OpenVINO Intermediate representation format](#Convert-the-model-to-OpenVINO-Intermediate-representation-format)
- [Use the OpenVINO IR model to run an inference](#Use-the-OpenVINO-IR-model-to-run-an-inference)


## Prerequisites[back to top ⬆️](#Table-of-contents:)


In [ ]:
%pip install -q --extra-index-url https://download.pytorch.org/whl/cpu torch torchvision
%pip install -q  "openvino>=2023.1.0"

## Get a test image
[back to top ⬆️](#Table-of-contents:)
First of all lets get a test image from an open dataset. 

In [ ]:
import urllib.request

from torchvision.io import read_image
import torchvision.transforms as transforms


img_path = 'cats_image.jpeg'
urllib.request.urlretrieve(
    url='https://huggingface.co/datasets/huggingface/cats-image/resolve/main/cats_image.jpeg',
    filename=img_path
)
image = read_image(img_path)
display(transforms.ToPILImage()(image))

## Get a pretrained model
[back to top ⬆️](#Table-of-contents:)
Torchvision provides a mechanism of [listing and retrieving available models](https://pytorch.org/vision/stable/models.html#listing-and-retrieving-available-models). 

In [ ]:
import torchvision.models as models

# List available models
all_models = models.list_models()
# List of models by type. Classification models are in the parent module.
classification_models = models.list_models(module=models)

print(classification_models)

We will use `convnext_tiny`. To get a pretrained model just use `models.get_model("convnext_tiny", weights='DEFAULT')` or a specific method of `torchvision.models` for this model using [default weights](https://pytorch.org/vision/stable/models/generated/torchvision.models.convnext_tiny.html#torchvision.models.ConvNeXt_Tiny_Weights) that is equivalent to `ConvNeXt_Tiny_Weights.IMAGENET1K_V1`. If you don't specify `weight` or specify `weights=None` it will be a random initialization. To get all available weights for the model you can call `weights_enum = models.get_model_weights("convnext_tiny")`, but there is only one for this model. You can find more information how to initialize pre-trained models [here](https://pytorch.org/vision/stable/models.html#initializing-pre-trained-models).

In [ ]:
model = models.convnext_tiny(weights=models.ConvNeXt_Tiny_Weights.DEFAULT)

## Define a preprocessing and prepare an input data
[back to top ⬆️](#Table-of-contents:)
You can use `torchvision.transforms` to make a preprocessing or use[preprocessing transforms from the model wight](https://pytorch.org/vision/stable/models.html#using-the-pre-trained-models).

In [ ]:
import torch


preprocess = models.ConvNeXt_Tiny_Weights.DEFAULT.transforms()

input_data = preprocess(image)
input_data = torch.stack([input_data], dim=0)

## Use the original model to run an inference[back to top ⬆️](#Table-of-contents:)


In [ ]:
outputs = model(input_data)

And print results

In [ ]:
import urllib.request


# download class number to class label mapping
imagenet_classes_file_path = "imagenet_2012.txt"
urllib.request.urlretrieve(
    url="https://storage.openvinotoolkit.org/repositories/openvino_notebooks/data/data/datasets/imagenet/imagenet_2012.txt",
    filename=imagenet_classes_file_path
)
imagenet_classes = open(imagenet_classes_file_path).read().splitlines()


def print_results(outputs: torch.Tensor):
    _, predicted_class = outputs.max(1)
    predicted_probability = torch.softmax(outputs, dim=1)[0, predicted_class].item()

    print(f"Predicted Class: {predicted_class.item()}")
    print(f"Predicted Label: {imagenet_classes[predicted_class.item()]}")
    print(f"Predicted Probability: {predicted_probability}")

In [ ]:
print_results(outputs)

## Convert the model to OpenVINO Intermediate representation format
[back to top ⬆️](#Table-of-contents:)

OpenVINO supports PyTorch through conversion to OpenVINO Intermediate Representation (IR) format. To take the advantage of OpenVINO optimization tools and features, the model should be converted using the OpenVINO Converter tool (OVC). The `openvino.convert_model` function provides Python API for OVC usage. The function returns the instance of the OpenVINO Model class, which is ready for use in the Python interface. However, it can also be saved on disk using `openvino.save_model` for future execution.

In [ ]:
from pathlib import Path

import openvino as ov 


ov_model_xml_path = Path('models/ov_convnext_model.xml')

if not ov_model_xml_path.exists():
    ov_model_xml_path.parent.mkdir(parents=True, exist_ok=True)
    converted_model = ov.convert_model(model, example_input=torch.randn(1, 3, 224, 224))
    # add transform to OpenVINO preprocessing converting
    ov.save_model(converted_model, ov_model_xml_path)
else:
    print(f"IR model {ov_model_xml_path} already exists.")

When the `openvino.save_model` function is used, an OpenVINO model is serialized in the file system as two files with `.xml` and `.bin` extensions. This pair of files is called OpenVINO Intermediate Representation format (OpenVINO IR, or just IR) and useful for efficient model deployment. OpenVINO IR can be loaded into another application for inference using the `openvino.Core.read_model` function. 

Select device from dropdown list for running inference using OpenVINO

In [ ]:
import ipywidgets as widgets

core = ov.Core()
device = widgets.Dropdown(
    options=core.available_devices + ["AUTO"],
    value='AUTO',
    description='Device:',
    disabled=False,
)

device

In [ ]:
core = ov.Core()

compiled_model = core.compile_model(ov_model_xml_path, device_name=device.value)

## Use the OpenVINO IR model to run an inference[back to top ⬆️](#Table-of-contents:)


In [ ]:
outputs = compiled_model(input_data)[0]
print_results(torch.from_numpy(outputs))